### RED CGAN:


In [4]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Embedding, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU

# Constants
IMG_ROWS, IMG_COLS, CHANNELS = 28, 28, 1
IMG_SHAPE = (IMG_ROWS, IMG_COLS, CHANNELS)
NOISE_DIM = 100
NUM_CLASSES = 10

# Load MNIST dataset
(x_train, y_train), (_, _) = mnist.load_data()

# Normalize and reshape images
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

# Rescale labels to one-hot vectors
y_train = np.eye(NUM_CLASSES)[y_train]

In [5]:
def build_generator():
    model = Sequential()
    
    model.add(Dense(256, input_dim=NOISE_DIM))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dense(np.prod(IMG_SHAPE), activation='tanh'))
    model.add(Reshape(IMG_SHAPE))

    noise = Input(shape=(NOISE_DIM,))
    label = Input(shape=(NUM_CLASSES,))
    label_embedding = Dense(NOISE_DIM)(label)

    model_input = multiply([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)

def build_discriminator():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=IMG_SHAPE, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))

    img = Input(shape=IMG_SHAPE)
    validity = model(img)

    return Model(img, validity)

In [6]:
# Build the generator
generator = build_generator()

# Build the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Freeze discriminator's weights during generator training
discriminator.trainable = False

# Build the CGAN by connecting generator and discriminator
noise = Input(shape=(NOISE_DIM,))
label = Input(shape=(NUM_CLASSES,))
img = generator([noise, label])
validity = discriminator(img)

# Compile the CGAN model
cgan = Model([noise, label], validity)
cgan.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
# Training
BATCH_SIZE = 32
EPOCHS = 200
SAMPLE_INTERVAL = 10

# Adversarial ground truths
valid = np.ones((BATCH_SIZE, 1))
fake = np.zeros((BATCH_SIZE, 1))

for epoch in range(EPOCHS):
    # Select a random batch of images
    idx = np.random.randint(0, x_train.shape[0], BATCH_SIZE)
    real_images = x_train[idx]
    labels = y_train[idx]

    # Generate a batch of fake images
    noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
    gen_images = generator.predict([noise, labels])

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_images, valid)
    d_loss_fake = discriminator.train_on_batch(gen_images, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
    g_loss = cgan.train_on_batch([noise, labels], valid)

    # Print progress
    if epoch % SAMPLE_INTERVAL == 0:
        print(f"Epoch {epoch}/{EPOCHS} | Discriminator loss: {d_loss[0]:.4f}, Generator loss: {g_loss:.4f}")

1/1 [==============================] - 0s 381ms/step
Epoch 0/200 | Discriminator loss: 0.6992, Generator loss: 0.6336
1/1 [==============================] - 0s 31ms/step
Epoch 10/200 | Discriminator loss: 0.3254, Generator loss: 0.2833
1/1 [==============================] - 0s 37ms/step
Epoch 20/200 | Discriminator loss: 0.1440, Generator loss: 0.4260
1/1 [==============================] - 0s 27ms/step
Epoch 30/200 | Discriminator loss: 0.0109, Generator loss: 1.1474
1/1 [==============================] - 0s 25ms/step
Epoch 40/200 | Discriminator loss: 0.0032, Generator loss: 0.5192
1/1 [==============================] - 0s 24ms/step
Epoch 50/200 | Discriminator loss: 0.0174, Generator loss: 0.7481
1/1 [==============================] - 0s 30ms/step


In [ ]:
import matplotlib.pyplot as plt

# Generate synthetic images
num_examples = 10
noise = np.random.normal(0, 1, (num_examples, NOISE_DIM))
labels = np.eye(NUM_CLASSES)[np.arange(num_examples) % NUM_CLASSES]
generated_images = generator.predict([noise, labels])

# Rescale generated images to [0, 1]
generated_images = 0.5 * generated_images + 0.5

# Plot generated images
fig, axs = plt.subplots(1, num_examples, figsize=(10, 2))
fig.suptitle('Generated Images', fontsize=14, fontweight='bold')
for i in range(num_examples):
    axs[i].imshow(generated_images[i, :, :, 0], cmap='gray')
    axs[i].axis('off')
plt.show()